In [14]:
import openturns as ot
import importlib.util
import sys
import os
import pandas as pd
from matplotlib import pylab as plt
import openturns.viewer as otv

# Define constants
MODULE_NAME = "FloodModel"
FILE_PATH = "examples/Water.py"
OUTPUT_DIR = "results"
OUTPUT_EXPECTATION_CSV = os.path.join(OUTPUT_DIR, "expectation_convergence.csv")
OUTPUT_GRID_CSV = os.path.join(OUTPUT_DIR, "grid.csv")
OUTPUT_FIRST_ORDER_SOBOL_CSV = os.path.join(OUTPUT_DIR, "first_order_sobol_indices.csv")
OUTPUT_TOTAL_ORDER_SOBOL_CSV = os.path.join(OUTPUT_DIR, "total_order_sobol_indices.csv")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load the flood model
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"File {FILE_PATH} does not exist.")

spec = importlib.util.spec_from_file_location(MODULE_NAME, FILE_PATH)
if spec is None:
    raise ImportError(f"Could not load the module from {FILE_PATH}.")

module = importlib.util.module_from_spec(spec)
sys.modules[MODULE_NAME] = module
spec.loader.exec_module(module)

function_of_interest, problem = module.model, module.problem

# Create distributions
distributions = ot.DistributionCollection()
for dist_info in problem['distributions']:
    dist_type = dist_info['type']
    params = dist_info['params']
    if dist_type == 'Uniform':
        distributions.add(ot.Uniform(*params))
    elif dist_type == 'Normal':
        distributions.add(ot.Normal(*params))
    elif dist_type == 'LogNormalMuSigma':
        distributions.add(ot.ParametrizedDistribution(ot.LogNormalMuSigma(*params)))
    elif dist_type == 'LogNormal':
        distributions.add(ot.LogNormal(*params))
    elif dist_type == 'Beta':
        distributions.add(ot.Beta(*params))
    elif dist_type == 'Gumbel':
        distributions.add(ot.Gumbel(*params))
    elif dist_type == 'Triangular':
        distributions.add(ot.Triangular(*params))
    else:
        raise ValueError(f"Unsupported distribution type: {dist_type}")

distribution = ot.ComposedDistribution(distributions)
input_names = problem['names']

# Define the OpenTURNS model
ot_model = ot.PythonFunction(problem['num_vars'], 1, function_of_interest)

# Draw the function
n = 10000
sampleX = distribution.getSample(n)
sampleY = ot_model(sampleX)

# Save data used in plotXvsY to CSV
sampleX.exportToCSVFile(os.path.join(OUTPUT_DIR, "X.csv"), ",")
sampleY.exportToCSVFile(os.path.join(OUTPUT_DIR, "Y.csv"), ",")

X = pd.read_csv(os.path.join(OUTPUT_DIR, 'X.csv'))
Y = pd.read_csv(os.path.join(OUTPUT_DIR, 'Y.csv'))
X.columns = problem['names']

grid_df = pd.concat([Y, X], axis=1)
grid_df.to_csv(OUTPUT_GRID_CSV, index=False)

# Estimate the Sobol' indices
size = 1000
sie = ot.SobolIndicesExperiment(distribution, size)
inputDesign = sie.generate()
inputDesign.setDescription(input_names)
outputDesign = ot_model(inputDesign)

sensitivityAnalysis = ot.SaltelliSensitivityAlgorithm(inputDesign, outputDesign, size)


graph = sensitivityAnalysis.draw()
view = otv.View(graph)
output_visual_dir = "results_visual/"
plt.savefig(os.path.join(output_visual_dir, "sobol_indices.png"))

# Create DataFrames for Sobol indices
rows = str(sensitivityAnalysis.getFirstOrderIndicesInterval()).split('\n')
data = [tuple(map(float, row.strip('[]').split(','))) for row in rows]
df = pd.DataFrame(data, columns=['Upper Bound', 'Lower Bound'])
new_df = pd.DataFrame({'Sobol Index': list(map(float, str(sensitivityAnalysis.getFirstOrderIndices()).strip('[]').split(',')))})
first_order_df = pd.concat([new_df, df], axis=1)

rows = str(sensitivityAnalysis.getTotalOrderIndicesInterval()).split('\n')
data = [tuple(map(float, row.strip('[]').split(','))) for row in rows]
df = pd.DataFrame(data, columns=['Upper Bound', 'Lower Bound'])
new_df = pd.DataFrame({'Sobol Index': list(map(float, str(sensitivityAnalysis.getTotalOrderIndices()).strip('[]').split(',')))})
total_order_df = pd.concat([new_df, df], axis=1)

# Save Sobol indices to CSV
first_order_df.to_csv(OUTPUT_FIRST_ORDER_SOBOL_CSV, index=False)
total_order_df.to_csv(OUTPUT_TOTAL_ORDER_SOBOL_CSV, index=False)

# Define the input distribution
input_vector = ot.RandomVector(distribution)

# The output vector is a CompositeRandomVector
output_vector = ot.CompositeRandomVector(ot_model, input_vector)

# Define the algorithm for expectation convergence
algo = ot.ExpectationSimulationAlgorithm(output_vector)
algo.setMaximumOuterSampling(8000)
algo.setBlockSize(1)
algo.setCoefficientOfVariationCriterionType("NONE")

# Run the algorithm and store the result
algo.run()
result = algo.getResult()

# Draw the convergence history and save the convergence data to a CSV file
graphConvergence = algo.drawExpectationConvergence()
data = graphConvergence.getDrawable(0).getData()
sample_sizes = data[:, 0]
mean_estimates = data[:, 1]

# Compute standard deviations for the mean estimates
standard_deviations = result.getStandardDeviation()

# Calculate confidence intervals
z_value = 1.96  # For a 95% confidence interval
lower_bounds = mean_estimates - z_value * standard_deviations
upper_bounds = mean_estimates + z_value * standard_deviations

df = pd.DataFrame({
    "Sample Size": [point[0] for point in sample_sizes],
    "Mean Estimate": [point[0] for point in mean_estimates],
    "Lower Bound": [point[0] for point in lower_bounds],
    "Upper Bound": [point[0] for point in upper_bounds]
})
df.to_csv(OUTPUT_EXPECTATION_CSV, index=False)

# Function to save correlation coefficients to CSV
def save_correlation_to_csv(corr_data, input_names, method, file_path):
    data = {
        'Variable': f"[{','.join(input_names)}]",
        'Correlation_Coefficient': list(corr_data)
    }
    df = pd.DataFrame(data)
    df.to_csv(file_path, index=False)

# Perform correlation analysis and save results to CSV
corr_analysis = ot.CorrelationAnalysis(sampleX, sampleY)

methods = {
    "PCC": corr_analysis.computePCC,
    "PRCC": corr_analysis.computePRCC,
    "SRC": corr_analysis.computeSRC,
    "SRRC": corr_analysis.computeSRRC,
    "Pearson": corr_analysis.computePearsonCorrelation,
    "Spearman": corr_analysis.computeSpearmanCorrelation,
}

for method, func in methods.items():
    indices = func()
    save_correlation_to_csv(indices, input_names, method, os.path.join(OUTPUT_DIR, f"{method}_coefficients.csv"))

# Combine CSV files for correlation coefficients
def combine_csv_files(input_dir, output_file):
    combined_df = None
    
    for file in os.listdir(input_dir):
        if file.endswith('_coefficients.csv'):
            method = file.replace('_coefficients.csv', '')
            df = pd.read_csv(os.path.join(input_dir, file))
            df = df.rename(columns={'Correlation_Coefficient': method})
            if combined_df is None:
                combined_df = df
            else:
                combined_df[method] = df[method]
    
    combined_df.to_csv(output_file, index=False)

combine_csv_files(OUTPUT_DIR, os.path.join(OUTPUT_DIR, 'combined_coefficients.csv'))


ImportError: Could not load the module from examples/Water.py.

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.ticker as ticker

# Ensure the output directory exists
output_visual_dir = "results_visual"
os.makedirs(output_visual_dir, exist_ok=True)

# Function to plot correlation coefficients
def plot_correlation_coefficients():
    # Load the data
    df = pd.read_csv('results/combined_coefficients.csv')

    # Extract the first 'Variable' entry and use it to create x-ticks
    xticks = df['Variable'][0].strip('[]').replace("'", "").split(',')

    # Convert the 'Variable' column to a unique identifier (e.g., just use the index for simplicity)
    df['Variable'] = df.index

    # Set the 'Variable' column as the index
    df.set_index('Variable', inplace=True)

    # Plot the data
    ax = df.plot(kind='bar', figsize=(12, 8))
    plt.xlabel('Variable Group Index')
    plt.ylabel('Correlation Coefficient')
    plt.title('Comparison of Correlation Coefficients Across Variables')
    plt.legend(title='Method')

    # Manually set the x-tick labels
    ax.set_xticklabels(xticks, rotation=45, ha='right')
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(os.path.join(output_visual_dir, "correlation_coefficients.png"))
    plt.close()

# Function to plot expectation convergence
def plot_expectation_convergence():
    # Load the CSV file
    csv_file = "results/expectation_convergence.csv"
    data = pd.read_csv(csv_file)

    # Plot the data
    plt.figure(figsize=(10, 6))

    # Plot the mean estimates
    plt.plot(data['Sample Size'], data['Mean Estimate'], label='Mean Estimate', color='blue')

    # Plot the confidence intervals
    plt.fill_between(np.array(data['Sample Size'], dtype=float), np.array(data['Lower Bound'], dtype=float), np.array(data['Upper Bound'], dtype=float), color='blue', alpha=0.2, label='95% Confidence Interval')

    # Adding titles and labels
    plt.title('Mean Estimate Convergence with Confidence Intervals')
    plt.xlabel('Sample Size')
    plt.ylabel('Mean Estimate')
    plt.legend()
    plt.grid(True)

    # Save the plot
    output_image_path = os.path.join(output_visual_dir, "mean_estimate_convergence_plot.png")
    plt.savefig(output_image_path)
    plt.close()

# Function to plot grid data
def plot_grid_data():
    # Load the CSV file
    csv_file = 'results/grid.csv'
    df = pd.read_csv(csv_file)

    # Extract the columns
    y = df['y0']
    X = df.drop(columns=['y0'])

    # Get the column names
    X_names = X.columns

    # Define the dimensions
    dimX = X.shape[1]

    # Create a grid of plots
    fig, axes = plt.subplots(1, dimX, figsize=(15, 5))

    # Plot each subplot
    for j in range(dimX):
        ax = axes[j] if dimX > 1 else axes
        ax.scatter(X.iloc[:, j], y, alpha=0.5, s=5)
        ax.set_xlabel(X_names[j], fontsize=12, fontweight='bold')
        ax.tick_params(axis='x', rotation=90)  # Rotate x-ticks vertically
        if j == 0:
            ax.set_ylabel('y0')
        else:
            ax.set_ylabel("")
        ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))

    plt.tight_layout()
    
    # Save the plot
    plt.savefig(os.path.join(output_visual_dir, "grid_plot.png"))
    plt.close()



# Generate all plots
plot_correlation_coefficients()
plot_expectation_convergence()
plot_grid_data()
